In [ ]:
import MySQLdb

In [ ]:
db = MySQLdb.connect(user='root', passwd="password", db="f1")

# Analysis of F1 points over all seasons

I wanted to know whether F1 racedays are becoming more or less competitive. I had a feeling that we needed to look at the relative importance of qualifying vs. racedays, in terms of how many points per (effectively) won on each day. The thinking was that: if most points are "already won" in qualifying, teams would make less race-competitive cars (since they only need to be qualifying-competitive).

Let's have a look...

We'll use Pandas, obviously

In [ ]:
import pandas as pd

Fetch all race results from database

In [ ]:
c = db.cursor()

f1_history = pd.read_sql('select * from races', db, index_col='raceId')

Figure out how many points where on offer in each race (because this changes every now and then with rules changes, and some races are awarded half points for not reaching full race distance)

In [ ]:
race_points = pd.read_sql('select raceId, sum(points) from results group by raceId', db, index_col='raceId')

In [ ]:
f1_history = f1_history.join(race_points)

In [ ]:
race_results = pd.read_sql('select * from results', db, index_col='resultId')

We're going to define `quali_points` as the number of "points" that each qualified had, in each race. We do that by looking up the number of points awarded to the person who finished the race in the position the qualifier in question sat on the grid. I.e. the driver who qualifies into grid position 4 has "already won" (by the end of qualifying) the number of points that the driver who finished the race in fourth position won (by the end of the race).

In [ ]:
def quali_points(row, cursor):
    raceId = row['raceId']
    grid = row['grid']
    cursor.execute('''select points from results where raceId=%d and position=%d''' % (raceId, grid))
    points = cursor.fetchone()
    try:
        return float(points[0])
    except TypeError:
        return 0

In [ ]:
race_results['quali_points'] = race_results.apply(quali_points, axis=1, args=(c,))

Now we work out how many *extra* points each driver won during the race. This is how many race points they were awarded, minus how many we calculated were "already won" by their qualifying position.

In [ ]:
race_results['points_gained_on_sunday'] = race_results.points - race_results.quali_points

We set a floor of zero on this, so that we only count overtakes! We could count how many points changed hands overall, but I wanted to bias this a bit to look at how well incentivised overtaking is, which means how many points people win by heading forwards.

In [ ]:
race_results.points_gained_on_sunday = race_results.points_gained_on_sunday.clip_lower(0)

Now, look at each race and work out how many of all the points awarded that race weekend were "gained" on Sunday.

**We call this the `raceday_novelty` factor.**

In [ ]:
race_gains = race_results.groupby('raceId')

In [ ]:
race_gains = race_gains.sum()[['points_gained_on_sunday']]

In [ ]:
f1_history = f1_history.join(race_gains)

In [ ]:
f1_history.rename(columns={"sum(points)": "total_points_awarded"}, inplace=True)

In [ ]:
f1_history['raceday_novelty'] = f1_history.points_gained_on_sunday / f1_history.total_points_awarded

In [ ]:
f1_history.sort_values(by='date', inplace=True)

#### Let's have a look at the history of raceday novelty

In [ ]:
%pylab inline

In [ ]:
import numpy as np

We'll group this by championship year, so that we see the overall trend over history

I've also listed some major historical points (rules and formula changes) in F1's evolution (from [Wikipedia's F1 History](https://en.wikipedia.org/wiki/History_of_Formula_One) and some of the championship articles

In [ ]:
import datetime

In [ ]:
figsize(10, 5)
championships = f1_history.groupby('year').mean()
championships['champ_year'] = championships.index
championships['date'] = pd.to_datetime(championships.champ_year, format='%Y')

plt.plot(f1_history.date, 100 * f1_history.raceday_novelty, alpha=0.2, color='k')
plt.plot(championships.date, 100 * championships.raceday_novelty, color='#B52B21', lw=4)

key_markers = {
    1966: 'Engine capacity doubled',
    1967: 'Cosworth DFV V8',
    1968: 'Aero wings (and tobacco sponsorship)',
    1975: 'Transverse gearbox',
    1977: 'Ground effect',
    1981: 'Carbon fibre chassis',
    1983: 'Turbochargers',
    1987: 'Turbo restrictions',
    1989: '3.5 naturally aspirated only',
    1991: 'Traction control',
    1994: 'Safety era: less power, the Plank',
    1995: 'Strict aero regulations, 3 litre engines',
    1998: 'V10 era',
    2003: 'New points system',
    2005: 'Ford leaves F1',
    2006: '2.4l V8s',
    2007: 'Bridgestone only tyres',
    2009: 'KERS, Double diffusers.',
    2011: 'Team orders allowed',
    2014: 'Hybrid engines'
}

ax = plt.gca()

for year, innovation in key_markers.iteritems():
    d = datetime.datetime.strptime(str(year), '%Y')
    plt.text(d, 100, innovation, rotation=45, va='bottom', size='xx-small')
    plt.axvline(d, color='k', alpha=0.8)
xlabel('Championship year')
ylabel('% of points newly won on racedays')
tight_layout(rect=(0, 0, 0.98, 0.8))
savefig('F1 raceday novelty.png', dpi=300)